# Vizugy portal scraper

We start by scraping the latest available water level data from the website https://www.vizugy.hu/?mapData=VizmerceLista#mapData using the BeautifulSoup (bs4) library. This website provides information on water level measuring stations across Hungary. We extract the necessary data using the appropriate HTML selectors to ensure accuracy and reliability.

Once we have the latest available data, we proceed to iterate through each link on the page to access the historical water level data for each measuring station (hourly data).

In [ ]:
import pandas as pd 
import os
import tqdm
from datetime import datetime
from sqlalchemy import create_engine
from sqlalchemy.sql import text

In [ ]:
VIZUGY_WEBPAGE = 'https://www.vizugy.hu/'

In [ ]:
RUN_DATETIME = datetime.now()

Connect to target database and truncate stanging tables

In [ ]:
engine = create_engine( os.getenv("PG_URL") )

In [ ]:
with engine.connect() as conn:
    conn.execute(text("truncate table raw_list").execution_options(autocommit=True))
    conn.execute(text("truncate table raw_hourly_data").execution_options(autocommit=True))
    conn.commit()

Get the first table from the `VizmerceLista` site

In [ ]:
df = pd.read_html(f'{VIZUGY_WEBPAGE}?mapData=VizmerceLista#mapData', extract_links="body")
df = df[0]

All columns here are tuple typed. First is the value of the cell, second is the link (if it is a link).

In [ ]:
df

We simply split the tupe to `_val` and `_url` columns.

In [ ]:
for col in df.columns:
    if col == 'Vízmérce':
        df[[col, f'{col}_url']] = pd.DataFrame(df[col].to_list(), index=df.index)
    else:
        df[col] = df[col].apply(lambda x: x[0])


In [ ]:
df["load_dt"] = RUN_DATETIME

In [ ]:
df.to_sql('raw_list', con=engine, if_exists='append', index_label='id')

This is how the link look like for a subpage (Station page)

In [ ]:
df.iloc[0]["Vízmérce_url"]


Let's got through all subpage (station page) and collect the hourly table. All of this data will be available as `hourly_data`. Scraping these hundreds page took a while (5 mins or so).

In [ ]:
df_list = []

for index, row in tqdm.tqdm(df.iterrows(), total=len(df)): 
    df2 = pd.read_html(f'{VIZUGY_WEBPAGE}{df.iloc[index]["Vízmérce_url"]}',parse_dates=True)
    df2[1]["Vízmérce"] = df.iloc[index]["Vízmérce"]
    df2[1]["Vízfolyás"] = df.iloc[index]["Vízfolyás"]
    df2[1]["URL"] = df.iloc[index]["Vízmérce_url"]
    df_list.append(df2[1])

hourly_data = pd.concat(df_list)


In [ ]:
hourly_data["load_dt"] = RUN_DATETIME

In [ ]:
hourly_data

Save to postgres 

In [ ]:
hourly_data.to_sql('raw_hourly_data', con=engine, if_exists='append',
           index_label='id')

In [ ]:
merge_list = """
MERGE INTO gauging_stations AS target
USING (
  SELECT
    "Vízmérce" || '-' || "Vízfolyás" AS id,
    "Vízmérce" AS gauging_station,
    "Vízfolyás" AS waterflow,
    "Szelvény (fkm)" AS river_km,
    to_timestamp("Időpont", 'YYYY.MM.DD. HH24:MI') AS measure_date,
    "Vízmérce_url" AS vizallas_url,
    CASE WHEN "Vízállás (cm)" = '' THEN NULL ELSE "Vízállás (cm)"::float END as water_level,
    load_dt
  FROM
    raw_list
) AS source
ON (target.id = source.id)
WHEN MATCHED THEN
  UPDATE SET
    gauging_station = source.gauging_station,
    waterflow = source.waterflow,
    river_km = source.river_km,
    measure_date = source.measure_date,
    vizallas_url = source.vizallas_url,
    water_level = source.water_level,
    load_dt = source.load_dt
WHEN NOT MATCHED THEN
  INSERT (id, gauging_station, waterflow, river_km, measure_date, vizallas_url, water_level, load_dt)
  VALUES (source.id, source.gauging_station, source.waterflow, source.river_km, source.measure_date, source.vizallas_url, source.water_level, source.load_dt)

"""

In [ ]:
merge_hourly_data = """

 MERGE INTO hourly_data AS target
USING (
  SELECT
    "Vízmérce" || '-' ||  "Vízfolyás" || '-' ||"Időpont" id,
    "Vízmérce" || '-' || "Vízfolyás" AS gauging_station_id,
    "Vízmérce" AS gauging_station,
    "Vízfolyás" AS waterflow,
    to_timestamp("Időpont", 'YYYY.MM.DD. HH24:MI') AS measure_date,
    "Vízállás (cm)" AS water_level,
    "Vízhozam (m3/s)" AS water_discharge,
    load_dt
  FROM
    raw_hourly_data
) AS source
ON (target.id = source.id)
WHEN MATCHED AND COALESCE(target.water_level,-9999) <> COALESCE(source.water_level,-9999) OR  
                 COALESCE(target.water_discharge,-9999) <> COALESCE(source.water_discharge,-9999)
  THEN
  UPDATE SET
    gauging_station = source.gauging_station,
    gauging_station_id = source.gauging_station_id,
    waterflow = source.waterflow,
    measure_date = source.measure_date,
    water_level = source.water_level,
    water_discharge = source.water_discharge,
    load_dt = source.load_dt
WHEN NOT MATCHED THEN
  INSERT (id, gauging_station_id, gauging_station, waterflow, measure_date, water_level, water_discharge, load_dt)
  VALUES (source.id, source.gauging_station_id, source.gauging_station, source.waterflow, source.measure_date, source.water_level, source.water_discharge, source.load_dt)

"""

In [ ]:
with engine.connect() as conn:
    conn.execute(text(merge_list).execution_options(autocommit=True))
    conn.execute(text(merge_hourly_data).execution_options(autocommit=True))
    conn.commit()

In [ ]:
True